In [1]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import pandas as pd
import numpy as np
import re

In [108]:

# Read the CSV files into pandas DataFrames
df_active_upc = pd.read_csv('ActiveUPCMarketingInformation.csv')
df_upc_images = pd.read_csv('UPCImages.csv')

# Drop null values in the 'ItemDocumentValue' column
df_upc_images = df_upc_images.dropna(subset=['ItemDocumentValue'])

# Join the DataFrames on 'ItemSku'
merged_df = pd.merge(df_active_upc, df_upc_images, on='ItemSku', how='outer')

In [109]:
len(merged_df['ItemSku'].unique())

309826

In [110]:
merged_df.shape

(541809, 12)

In [111]:
image_yes_desc_yes = merged_df[(merged_df['ItemDocumentValue'].notnull()) & (merged_df['MarketingDetails'].notnull())]
image_yes_desc_no = merged_df[(merged_df['ItemDocumentValue'].notnull()) & (merged_df['MarketingDetails'].isnull())]
image_no_desc_yes= merged_df[(merged_df['ItemDocumentValue'].isnull()) & (merged_df['MarketingDetails'].notnull())]
image_no_desc_no = merged_df[(merged_df['ItemDocumentValue'].isnull()) & (merged_df['MarketingDetails'].isnull())]

image_yes_desc_yes['category'] = 1
image_yes_desc_no['category'] = 2
image_no_desc_yes['category'] = 3
image_no_desc_no['category'] = 4

C:\Users\gowth\AppData\Local\Temp\ipykernel_19196\743511628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_yes_desc_yes['category'] = 1
C:\Users\gowth\AppData\Local\Temp\ipykernel_19196\743511628.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_yes_desc_no['category'] = 2
C:\Users\gowth\AppData\Local\Temp\ipykernel_19196\743511628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [112]:
image_no_desc_yes

,ItemSku,UPCTypeName,ProductName,MarketingName,MarketingDetails,MarketingFeatures,UpdatedDate,ItemDocumentNote,ItemImageApprovalDate,ItemImageExpirationDate,ItemImageActiveDate,ItemDocumentValue,category
6,2410000140,UPCA,CHEEZ-IT WHITE CHEDDAR 12CT CADDY ...,"Cheez-It Cheese Crackers, White Cheddar, 12oz ...",Make snack time more fun with Cheez-It White C...,Light and crispy bite-sized snack crackers mad...,NaN,NaN,NaN,NaN,NaN,NaN,3
10,7797509556,UPCA,SNYDER VARIETY PACK PIECES 20CT ...,SNYDER VARIETY PACK PIECES 20CT,SNYDER VARIETY PACK PIECES 20CT,SNYDER VARIETY PACK PIECES 20CT,NaN,NaN,NaN,NaN,NaN,NaN,3
11,19556680261,UPCA,VINTAGE PBR TEE PABR008MSC BLK XL ...,VINTAGE PBR PABR008MSC BLACK MIN WASHED TEE XL,VINTAGE PBR PABR008MSC BLACK MIN WASHED TEE XL,VINTAGE PBR PABR008MSC BLACK MIN WASHED TEE XL,NaN,NaN,NaN,NaN,NaN,NaN,3
12,206117900000,EAN,CHICKEN BSB BROWN SUGAR/MAPLE WHISKEY-FP RW117...,Chicken BSB Brown Sugar/Maple Whiskey-FP,Chicken BSB Brown Sugar/Maple Whiskey-FP,Chicken BSB Brown Sugar/Maple Whiskey-FP,NaN,NaN,NaN,NaN,NaN,NaN,3
13,26117800000,UPCA,CHICKEN BSB BROWN SUGAR/MAPLE WHISKEY-FP RW117...,Chicken BSB Brown Sugar/Maple Whiskey-FP,Chicken BSB Brown Sugar/Maple Whiskey-FP,Chicken BSB Brown Sugar/Maple Whiskey-FP,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541802,70882029407,UPCA,ACTIVE 4-POINT MASSAGE ROLLER ...,ACTIVE 4-POINT MASSAGE ROLLER,Material: 70%PP+30%TPE Dimension: 8.46*1.96*12...,Material: 70%PP+30%TPE Dimension: 8.46*1.96*12...,NaN,NaN,NaN,NaN,NaN,NaN,3
541805,80685301164,UPCA,BOPPER STRIP B 3 AST ...,VENDOR TO CONFIRM,VENDOR TO CONFIRM,VENDOR TO CONFIRM,NaN,NaN,NaN,NaN,NaN,NaN,3
541806,80685301166,UPCA,BOPPER STRIP B 3 AST ...,VENDOR TO CONFIRM,VENDOR TO CONFIRM,VENDOR TO CONFIRM,NaN,NaN,NaN,NaN,NaN,NaN,3
541807,80685301167,UPCA,BOPPER STRIP B 3 AST ...,VENDOR TO CONFIRM,VENDOR TO CONFIRM,VENDOR TO CONFIRM,NaN,NaN,NaN,NaN,NaN,NaN,3


In [113]:
a = len(image_yes_desc_yes) 
b = len(image_yes_desc_no)
c = len(image_no_desc_yes) 
d = len(image_no_desc_no)
print(a, b, c, d)

334239 7817 52041 147712


In [152]:
# Get unique values from the 'ItemSku' column
unique_values_1 = image_yes_desc_yes['ItemSku'].unique()
unique_values_2 = image_yes_desc_no['ItemSku'].unique()
unique_values_3 = image_no_desc_yes['ItemSku'].unique()
unique_values_4 = image_no_desc_no['ItemSku'].unique()

# Select 10 unique values randomly
random_unique_values_1 = pd.Series(unique_values_1).sample(n=10, random_state=42).tolist()
random_unique_values_2 = pd.Series(unique_values_2).sample(n=10, random_state=42).tolist()
random_unique_values_3 = pd.Series(unique_values_3).sample(n=10, random_state=42).tolist()
random_unique_values_4 = pd.Series(unique_values_4).sample(n=10, random_state=42).tolist()



In [153]:
print(f"unique_values_1:{len(unique_values_1)}")
print(f"unique_values_2:{len(unique_values_2)}")
print(f"unique_values_3:{len(unique_values_3)}")
print(f"unique_values_4:{len(unique_values_4)}")

unique_values_1:107079
unique_values_2:3682
unique_values_3:51961
unique_values_4:147104


In [154]:
sku_df = pd.concat([image_yes_desc_yes[image_yes_desc_yes['ItemSku'].isin(random_unique_values_1)], 
                    image_yes_desc_no[image_yes_desc_no['ItemSku'].isin(random_unique_values_2)],
                   image_no_desc_yes[image_no_desc_yes['ItemSku'].isin(random_unique_values_3)],
                   image_no_desc_no[image_no_desc_no['ItemSku'].isin(random_unique_values_4)]])


In [155]:
sku_df.shape

(74, 13)

In [172]:
sku_df

,ItemSku,UPCTypeName,ProductName,MarketingName,MarketingDetails,MarketingFeatures,UpdatedDate,ItemDocumentNote,ItemImageApprovalDate,ItemImageExpirationDate,ItemImageActiveDate,ItemDocumentValue,category
61446,70882072796,UPCA,MEIJER DOG LEASH PINK LARGE ...,"Meijer Dog Leash, Pink, Large",Classic Style For Your Large Dog.,"Durable 1"" Nylon Webbing 5' Length",2021-03-15T08:30:07.300Z,upc_img_pth_frnt,2015-12-09,2099-12-31,2015-12-09,http://static.meijer.com/Media/007/08820/00708...,1
61447,70882072796,UPCA,MEIJER DOG LEASH PINK LARGE ...,"Meijer Dog Leash, Pink, Large",Classic Style For Your Large Dog.,"Durable 1"" Nylon Webbing 5' Length",2021-03-15T08:30:07.300Z,upc_img_pth_bk,2015-12-09,2099-12-31,2015-12-09,http://static.meijer.com/Media/007/08820/00708...,1
107603,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_frnt,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1
107604,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_bk,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1
107605,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_lft,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340327,71373316809,UPCA,MTA GRAPHIC TEE 50161 GREEN XS 5 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
380059,1071928353591,UPCB,MEIJER GREEK YOGURT 0% BLACK CHERRY 5.3 OZ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
409825,103076673930,UPCB,TUMS ULTRA MINT TABS 72 CT ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
475634,1060407901604,UPCB,PHILOSOPHY RASPBERRY SORBET BATH 3 IN 1 16 OZ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


# image_yes_desc_yes

In [123]:
ProductName = []
ItemDocumentValue = []
product_desc = []

temp_df = sku_df[sku_df['category'] == 1]
unique_values = temp_df['ItemSku'].unique()

for sku in unique_values:
    # Filter the DataFrame for the current SKU
    temp_df2 = temp_df[temp_df['ItemSku'] == sku]
    
    # Extract the ProductName for the current SKU
    product_name = temp_df2['ProductName'].iloc[0]  # Assuming there's only one ProductName per SKU
    ProductName.append(product_name.strip())
    
    # Extract the ItemDocumentValue for the current SKU
    item_values = temp_df2['ItemDocumentValue'].tolist()
    ItemDocumentValue.append(item_values)
    
    product_desc.append(temp_df2['MarketingFeatures'].iloc[0]) 

In [176]:
product_desc

['Durable 1" Nylon Webbing 5\' Length',
 'FIDGETS, BUT MAKE IT FASHION! Fashion Fidget dolls are styled in totally fierce fits that double as fidget toys! Each Fashion Fidget doll has 3 fidget features! Will your fidget fashionista pop, twist, pull, or spin?!',
 'Usage: Small Game / Upland / Recreational Shooting  Rounds per box: 25  Engineered Wads - Thicker wad segments for increased barrel protection  Xpert Steel Shot - High-performance, corrosion resistant. Meets federal and state lead free projectile regulations.',
 'Meijer Branded Mercury Free Formula Guaranteed fresh for up to 10 years',
 'Contains one (1) 7-ounce HERSHEY\'S KISSES Solid Milk Chocolate Giant Candy Giant solid milk chocolate that makes a perfect candy gift for birthdays, promotions, graduations, holidays and just-because moments Individually wrapped KISSES Milk Chocolate Candy that\'s kosher and gluten free packaged inside a cut-out box with a bow on top Give a giant HERSHEY\'S KISSES candy to any special people 

In [173]:
ProductName

['MEIJER DOG LEASH PINK LARGE',
 'FASHION FIDGETS',
 'WINCHESTER X-PERT 20GA GAME & TARGET 7 STEEL',
 'MEIJER POWERCELL BATTERY AA-48PK CLUB PACK',
 'HERSHEYS HOLIDAY GIANT KISS 7 OZ',
 'SPROUT STG 2 POUCH APPL/OATML/RAISIN/CINN 3.5 OZ',
 'BEECHNUT MELTIES PROBIOTIC APPLE CARROT MANGO',
 'SH SE PM NAILS MEET ME IN THE METAL',
 'BILL KNAPPS DUNKER CHOCOLATE',
 'PURELY INSPIRED MCT OIL FL 16 OZ']

In [174]:
ItemDocumentValue

[['http://static.meijer.com/Media/007/08820/0070882072796_0_A1C1_0600.png',
  'http://static.meijer.com/Media/007/08820/0070882072796_0_A7C1_0600.png'],
 ['https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A1C1_0600.jpg',
  'https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A7C1_0600.jpg',
  'https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A2C1_0600.jpg',
  'https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A8C1_0600.jpg',
  'https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A3C1_0600.jpg',
  'https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A9C1_0600.jpg'],
 ['http://static.meijer.com/Media/000/20892/0002089201902_0_A1C1_0600.png'],
 ['http://static.meijer.com/Media/007/13733/0071373329422_1_A1C1_0600.png'],
 ['http://static.meijer.com/Media/000/34000/0003400002505_1_A1C1_0600.png',
  'http://

In [127]:
import re
from syllapy import count as count_syllables

def count_words(text):
    words = re.findall(r'\b\w+\b', text)
    return len(words)

def count_sentences(text):
    sentences = re.split(r'[.!?]', text)
    return len(sentences)

def count_complex_words(text):
    words = re.findall(r'\b\w+\b', text)
    complex_words = [word for word in words if len(re.findall(r'[aeiouy]+', word.lower())) > 2]
    return len(complex_words)

def calculate_flesch_kincaid_grade_level(words, sentences, syllables):
    return 0.39 * (words / sentences) + 11.8 * (syllables) - 15.59

def calculate_gunning_fog_index(words, sentences, complex_words):
    return 0.4 * ((words / sentences) + 100 * (complex_words / words))




In [128]:
flesch_kincaid_grade_level = []
gunning_fog_index = []
for i in range(len(ProductName)):
    text = product_desc[i]
    num_words = count_words(text)
    num_sentences = count_sentences(text)
    num_complex_words = count_complex_words(text)

    # Counting actual number of syllables using syllapy
    total_syllables = sum(count_syllables(word) for word in re.findall(r'\b\w+\b', text))

    # Avg syllables per word
    avg_syllables_per_word = total_syllables / num_words

    FK_grade_level = calculate_flesch_kincaid_grade_level(num_words, num_sentences, avg_syllables_per_word)
    GF_index = calculate_gunning_fog_index(num_words, num_sentences, num_complex_words)
    flesch_kincaid_grade_level.append(FK_grade_level)
    gunning_fog_index.append(GF_index)

In [129]:
flesch_kincaid_grade_level

[2.4833333333333343,
 4.122222222222224,
 9.653529411764708,
 7.773333333333333,
 23.388235294117653,
 19.807272727272736,
 10.511666666666667,
 3.4666666666666686,
 4.661666666666669,
 13.450000000000003]

In [130]:
system_prompt2 = "You are a helpful AI Assistant. You are working for a national retailer whose Brand Voice is characterized as friendly, informative, and trustworthy. At the core of Retailers's brand lies a commitment to clarity and comprehension. They have generated a product descriptions and your job is to judge if the description generated is able to capture the attention, interest of the consumer. Also, judge if the description is able to cultivate desire and prompt the potential customer take action such as making a purchase. For each of these four: Attention, interest, desire and action give a score out of 10 in the format Attention: x/10 etc. The output should be the scores out of 10 only"

In [131]:
import requests
import json
api_key = "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"

In [132]:
response = [0]*len(product_desc)
for i in range(len(product_desc)):
    prompt2 = f"product description generated is {product_desc[i]}"
# Send the request with both "system" and "user" roles
    response[i] = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "HTTP-Referer": "YOUR_SITE_URL",  # Optional, replace YOUR_SITE_URL with your actual site URL.
            "X-Title": "YOUR_APP_NAME",  # Optional, replace YOUR_APP_NAME with your actual app name.
        },
        data=json.dumps({
            "model": "mistralai/mistral-7b-instruct:free",  # Optional
            "messages": [
                {"role": "system", "content": system_prompt2},
                {"role": "user", "content": prompt2}
            ]
        })
    )


In [133]:
aida_framework = []
for i in range(len(ProductName)):
    aida_framework.append(response[i].json()['choices'][0]['message']['content'])

In [134]:
import re

# Provided text
attention = []
interest = []
desire = []
action = []

for i in range(len(ProductName)):
    text = aida_framework[i]
# Extracting Attention, Interest, Desire, and Action scores
    attention.append(int(re.search(r'Attention: (\d+)/10', text).group(1)))
    interest.append(int(re.search(r'Interest: (\d+)/10', text).group(1)))
    desire.append(int(re.search(r'Desire: (\d+)/10', text).group(1)))
    action.append(int(re.search(r'Action: (\d+)/10', text).group(1)))


In [140]:
data = {
    'ItemSku': unique_values,
    'Product Description': product_desc,
    'flesch_kincaid_grade_level': flesch_kincaid_grade_level,
    'gunning_fog_index': gunning_fog_index,
    'attention': attention,
    'interest': interest,
    'desire': desire,
    'action': action,
}

# Create DataFrame
df = pd.DataFrame(data)
# Print DataFrame
df.head()


,ItemSku,Product Description,flesch_kincaid_grade_level,gunning_fog_index,attention,interest,desire,action
0,70882072796,"Durable 1"" Nylon Webbing 5' Length",2.483333,9.066667,6,7,5,4
1,77117111481,"FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",4.122222,5.733333,7,6,5,5
2,2089201902,Usage: Small Game / Upland / Recreational Shoo...,9.653529,17.474510,7,5,6,5
3,71373329422,Meijer Branded Mercury Free Formula Guaranteed...,7.773333,14.800000,7,5,6,4
4,3400002505,Contains one (1) 7-ounce HERSHEY'S KISSES Soli...,23.388235,26.282353,7,6,5,4


In [164]:

# Define function to determine pass/fail
def pass_fail(row):
    if (row['flesch_kincaid_grade_level'] < 15 and
        row['gunning_fog_index'] < 20 and
        row['attention'] > 5 and
        row['interest'] > 5 and
        row['desire'] > 5 and
        row['action'] > 5):
        return 'Pass'
    else:
        return 'Fail'

# Apply the function to create 'Pass/Fail' column
df['Pass/Fail'] = df.apply(pass_fail, axis=1)

In [165]:
df

,ItemSku,Product Description,flesch_kincaid_grade_level,gunning_fog_index,attention,interest,desire,action,Pass/Fail
0,70882072796,"Durable 1"" Nylon Webbing 5' Length",2.483333,9.066667,6,7,5,4,Fail
1,77117111481,"FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",4.122222,5.733333,7,6,5,5,Fail
2,2089201902,Usage: Small Game / Upland / Recreational Shoo...,9.653529,17.474510,7,5,6,5,Fail
3,71373329422,Meijer Branded Mercury Free Formula Guaranteed...,7.773333,14.800000,7,5,6,4,Fail
4,3400002505,Contains one (1) 7-ounce HERSHEY'S KISSES Soli...,23.388235,26.282353,7,6,5,4,Fail
5,81851201209,For babies 6 months and up USDA certified orga...,19.807273,25.321212,8,7,6,5,Fail
6,5220004105,Stage 3: for 8 months and up Made with just 6 ...,10.511667,15.084848,7,8,6,7,Pass
7,361630401223,"24 nails, file, cuticle stick, prep pad Adhesi...",3.466667,12.400000,6,5,4,3,Fail
8,822910111,Chocolate Dunkers 12oz.,4.661667,13.933333,6,5,4,3,Fail
9,63165680117,Supports a Ketogenic Diet & is Paleo Diet Frie...,13.450000,14.000000,7,7,8,7,Pass


In [167]:
updated_df = pd.merge(sku_df, df[['ItemSku', 'Pass/Fail']], on='ItemSku', how='left')

# Fill missing values in 'Pass/Fail' column with 'Fail'
updated_df['Pass/Fail'].fillna('Fail', inplace=True)

In [168]:
updated_df

,ItemSku,UPCTypeName,ProductName,MarketingName,MarketingDetails,MarketingFeatures,UpdatedDate,ItemDocumentNote,ItemImageApprovalDate,ItemImageExpirationDate,ItemImageActiveDate,ItemDocumentValue,category,Pass/Fail
0,70882072796,UPCA,MEIJER DOG LEASH PINK LARGE ...,"Meijer Dog Leash, Pink, Large",Classic Style For Your Large Dog.,"Durable 1"" Nylon Webbing 5' Length",2021-03-15T08:30:07.300Z,upc_img_pth_frnt,2015-12-09,2099-12-31,2015-12-09,http://static.meijer.com/Media/007/08820/00708...,1,Fail
1,70882072796,UPCA,MEIJER DOG LEASH PINK LARGE ...,"Meijer Dog Leash, Pink, Large",Classic Style For Your Large Dog.,"Durable 1"" Nylon Webbing 5' Length",2021-03-15T08:30:07.300Z,upc_img_pth_bk,2015-12-09,2099-12-31,2015-12-09,http://static.meijer.com/Media/007/08820/00708...,1,Fail
2,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_frnt,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1,Fail
3,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_bk,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1,Fail
4,77117111481,UPCA,FASHION FIDGETS ...,FASHION FIDGETS,"Fidget toy, but make it fashion! Introducing F...","FIDGETS, BUT MAKE IT FASHION! Fashion Fidget d...",2022-09-17T05:47:01.001Z,upc_img_pth_lft,2022-09-16,2199-12-31,2022-09-16,https://www.meijer.com/content/dam/meijer/prod...,1,Fail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,71373316809,UPCA,MTA GRAPHIC TEE 50161 GREEN XS 5 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Fail
70,1071928353591,UPCB,MEIJER GREEK YOGURT 0% BLACK CHERRY 5.3 OZ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Fail
71,103076673930,UPCB,TUMS ULTRA MINT TABS 72 CT ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Fail
72,1060407901604,UPCB,PHILOSOPHY RASPBERRY SORBET BATH 3 IN 1 16 OZ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Fail


# image_yes_desc_no

In [177]:
ProductName = []
ItemDocumentValue = []
product_desc = []

temp_df = updated_df[((updated_df['category'] == 1) | (updated_df['category'] == 2)) & (updated_df['Pass/Fail'] == 'Fail')]
unique_values = temp_df['ItemSku'].unique()

for sku in unique_values:
    # Filter the DataFrame for the current SKU
    temp_df2 = temp_df[temp_df['ItemSku'] == sku]
    
    # Extract the ProductName for the current SKU
    product_name = temp_df2['ProductName'].iloc[0]  # Assuming there's only one ProductName per SKU
    ProductName.append(product_name.strip())
    
    # Extract the ItemDocumentValue for the current SKU
    item_values = temp_df2['ItemDocumentValue'].tolist()
    ItemDocumentValue.append(item_values)
    
    product_desc.append(temp_df2['MarketingFeatures'].iloc[0]) 

In [179]:
# Create an Image Analysis client
client = ImageAnalysisClient(
    endpoint='https://ipmicro-ocr.cognitiveservices.azure.com/',
    credential=AzureKeyCredential('2c5aad069013400f81faae0c05fd5871')
)

In [181]:
captions = [0] * len(ItemDocumentValue)
desc = [0] * len(ItemDocumentValue)

for i in range(len(ItemDocumentValue)):
    temp_des = ''
    caption = ''
    
    for url in ItemDocumentValue[i]:
        try:
            result = client.analyze_from_url(
                image_url=url,
                visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ, VisualFeatures.TAGS],
                gender_neutral_caption=True,  # Optional (default is False)
            )

            if result.caption is not None:
                caption += " | " + result.caption.text

            if result.read is not None:
                try:
                    for line in result.read.blocks[0].lines:
                        temp_des += ' ' + line.text
                except:
                    pass  # Handling exception if blocks or lines are not present

        except Exception as e:
            print(f"Error processing image URL {url} {e}")
            continue  # Skip to the next URL if an error occurs

    desc[i] = temp_des
    captions[i] = caption


Error processing image URL https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A1C1_0600.jpg (InvalidRequest) The provided image url is not accessible.
Code: InvalidRequest
Message: The provided image url is not accessible.
Error processing image URL https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A7C1_0600.jpg (InvalidRequest) The provided image url is not accessible.
Code: InvalidRequest
Message: The provided image url is not accessible.
Error processing image URL https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A2C1_0600.jpg (InvalidRequest) The provided image url is not accessible.
Code: InvalidRequest
Message: The provided image url is not accessible.
Error processing image URL https://www.meijer.com/content/dam/meijer/product/0771/17/1114/81/0771171114819_0_A8C1_0600.jpg (InvalidRequest) The provided image url is not accessible.
Code: InvalidRequest
Message: The provided image url is 

In [203]:
import requests
import json
api_key = "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"

In [183]:
system_prompt = "You are a helpful AI Assistant. You are working for a national retailer whose Brand Voice is characterized as friendly, informative, and trustworthy. At the core of retailer's brand lies a commitment to clarity and comprehension. When drafting product descriptions, it is essential to directly mirror the product's features as they are officially listed. Every feature should be presented precisely and understandably in the description to avoid any confusion. For example, if a product is listed with “Organic Cotton, 300 Thread Count,” it should be stated exactly as such, not as Natural Cotton, 300TC. Generate the product descriptions.Also give the confidence score out of 100% for the response should be in the format \"Confidence score: x%\""

In [184]:
response = [0]*len(ProductName)
for i in range(len(ProductName)):
    prompt = f"product name is {ProductName[i]} and information captured from the images are {desc[i]}. The caption from a Vision AI described the image as {captions[i]}"
    # Send the request with both "system" and "user" roles
    response[i] = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "HTTP-Referer": "YOUR_SITE_URL",  # Optional, replace YOUR_SITE_URL with your actual site URL.
            "X-Title": "YOUR_APP_NAME",  # Optional, replace YOUR_APP_NAME with your actual app name.
        },
        data=json.dumps({
            "model": "mistralai/mistral-7b-instruct:free",  # Optional
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        })
    )



In [185]:
content = []
for i in range(len(ProductName)):
    content.append(response[i].json()['choices'][0]['message']['content'])

In [186]:
confidence_scores = []
for i in range(len(ProductName)):
    # Using regular expression to extract the confidence score
    confidence_score_pattern = r'Confidence score: (\d+)%'
    confidence_score_match = re.search(confidence_score_pattern, content[i])

    if confidence_score_match:
        confidence_scores.append(confidence_score_match.group(1))
        
    else:
        confidence_scores.append("Not found")


# Relevance: Cosine Similarity

In [187]:
from collections import Counter
import math

def preprocess_text(text):
    # Tokenize the text into words, remove punctuation, and convert to lowercase
    words = text.lower().split()
    words = [word.strip('.,?!') for word in words]
    return words

def compute_cosine_similarity(vector1, vector2):
    # Compute the dot product of the two vectors
    dot_product = sum(vector1[key] * vector2.get(key, 0) for key in vector1)
    
    # Compute the magnitude of each vector
    magnitude1 = math.sqrt(sum(val ** 2 for val in vector1.values()))
    magnitude2 = math.sqrt(sum(val ** 2 for val in vector2.values()))
    
    # Compute the cosine similarity
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    return cosine_similarity




In [188]:
cosine_similarity = []
for i in range(len(ProductName)):
    prompt = f"product name is {ProductName[i]} and information captured from the images are {desc[i]}. The caption from a Vision AI described the image as {captions[i]}. Also give the confidence score out of 100% of the response in the format, Confidence score: xx%"
    
# Define the two paragraphs
    paragraph1 = prompt
    paragraph2 = content[i]

    # Preprocess the paragraphs
    words1 = preprocess_text(paragraph1)
    words2 = preprocess_text(paragraph2)

    # Compute word frequencies for each paragraph
    word_freq1 = Counter(words1)
    word_freq2 = Counter(words2)

    # Compute cosine similarity
    similarity = compute_cosine_similarity(word_freq1, word_freq2)
    cosine_similarity.append(similarity)

In [189]:
prompt1 = f"product name is {ProductName[1]} and information captured from the images are {desc[1]}. The caption from a Vision AI described the image as {captions[1]}. Also give the confidence score out of 100% of the response in the format, Confidence score: xx%"
prompt1

'product name is FASHION FIDGETS and information captured from the images are . The caption from a Vision AI described the image as . Also give the confidence score out of 100% of the response in the format, Confidence score: xx%'

In [190]:
content[1]

"Fashion Fidgets are trendy and functional accessories designed to keep you on the go. With a range of colors and styles, you can find the perfect Fidget to suit your personality and preferences. Made with 100% natural, organic cotton, these Fidgets are both comfortable and durable. Each Fidget is carefully crafted with a 300 thread count, ensuring that it is both comfortable to wear and long-lasting. Whether you're busy at work, studying, or on the go, Fashion Fidgets are the perfect solution for those who need something to keep them occupied. With a range of stylish colors available, there is a Fidget to suit everyone. Trust us - you won't be able to put them down! Confidence score: 100%"

# Clarity

In [191]:
import re
from syllapy import count as count_syllables

def count_words(text):
    words = re.findall(r'\b\w+\b', text)
    return len(words)

def count_sentences(text):
    sentences = re.split(r'[.!?]', text)
    return len(sentences)

def count_complex_words(text):
    words = re.findall(r'\b\w+\b', text)
    complex_words = [word for word in words if len(re.findall(r'[aeiouy]+', word.lower())) > 2]
    return len(complex_words)

def calculate_flesch_kincaid_grade_level(words, sentences, syllables):
    return 0.39 * (words / sentences) + 11.8 * (syllables) - 15.59

def calculate_gunning_fog_index(words, sentences, complex_words):
    return 0.4 * ((words / sentences) + 100 * (complex_words / words))




In [192]:
flesch_kincaid_grade_level = []
gunning_fog_index = []
for i in range(len(ProductName)):
    text = content[i]
    num_words = count_words(text)
    num_sentences = count_sentences(text)
    num_complex_words = count_complex_words(text)

    # Counting actual number of syllables using syllapy
    total_syllables = sum(count_syllables(word) for word in re.findall(r'\b\w+\b', text))

    # Avg syllables per word
    avg_syllables_per_word = total_syllables / num_words

    FK_grade_level = calculate_flesch_kincaid_grade_level(num_words, num_sentences, avg_syllables_per_word)
    GF_index = calculate_gunning_fog_index(num_words, num_sentences, num_complex_words)
    flesch_kincaid_grade_level.append(FK_grade_level)
    gunning_fog_index.append(GF_index)

In [193]:
flesch_kincaid_grade_level

[8.381555555555558,
 7.764948347107438,
 9.541230769230769,
 11.439917525773197,
 7.936410256410259,
 11.6252380952381,
 7.016030927835054,
 8.480776119402986,
 5.490408602150541,
 10.327368421052633,
 4.961904761904762,
 7.51089026915114,
 8.477289377289381,
 5.812000000000001,
 7.502857142857145,
 9.227500000000003,
 9.685454545454544,
 7.929000000000002]

# AIDA Framework

In [194]:
system_prompt2 = "You are a helpful AI Assistant. You are working for a national retailer. They have generated a product descriptions and your job is to judge if the description generated is able to capture the attention, interest of the consumer. Also, judge if the description is able to cultivate desire and prompt the potential customer take action such as making a purchase. For each of these four: Attention, interest, desire and action give a score out of 10 in the format Attention: x/10 etc. Do not generate any response other than the scores for those four attributes. The output should be the scores out of 10"

In [235]:
response = [0]*len(content)
for i in range(len(content)):
    prompt2 = f"product description generated is {content[i]}"
# Send the request with both "system" and "user" roles
    response[i] = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "HTTP-Referer": "YOUR_SITE_URL",  # Optional, replace YOUR_SITE_URL with your actual site URL.
            "X-Title": "YOUR_APP_NAME",  # Optional, replace YOUR_APP_NAME with your actual app name.
        },
        data=json.dumps({
            "model": "mistralai/mistral-7b-instruct:free",  # Optional
            "messages": [
                {"role": "system", "content": system_prompt2},
                {"role": "user", "content": prompt2}
            ]
        })
    )


In [243]:
aida_framework = []

# Assuming response is a list of response objects
for resp in response:
    # Check if the response was successful (status code 200)
    if resp.status_code == 200:
        try:
            # Extract and append text content
            content = resp.json()['choices'][0]['message']['content']
            aida_framework.append(content)
        except (KeyError, IndexError):
            # Handle the case where the JSON structure doesn't match expected format
            print("Error: Could not extract content from response:", resp)
            aida_framework.append(None)  # Placeholder for failed extraction
    else:
        # Handle the case where the response status code indicates an error
        print("Error: Response status code was not 200:", resp.status_code)
        aida_framework.append(None)  # Placeholder for failed extraction


Error: Response status code was not 200: 520
Error: Response status code was not 200: 524


In [254]:
import re

# Provided text
attention = []
interest = []
desire = []
action = []

for i in range(len(aida_framework)):
    text = aida_framework[i]
    
    # Check if text is not None and is a string
    if text is not None and isinstance(text, str):
        try:
            # Extracting Attention, Interest, Desire, and Action scores
            attention_score = int(re.search(r'Attention: (\d+)/10', text).group(1))
            interest_score = int(re.search(r'Interest: (\d+)/10', text).group(1))
            desire_score = int(re.search(r'Desire: (\d+)/10', text).group(1))
            action_score = int(re.search(r'Action: (\d+)/10', text).group(1))
            
            # Append scores to respective lists
            attention.append(attention_score)
            interest.append(interest_score)
            desire.append(desire_score)
            action.append(action_score)
        except AttributeError:
            # Handle cases where the regex search fails
            print("Error: Regex search failed for text:", text)
            
            # Append None as a placeholder for failed extraction
            attention.append(None)
            interest.append(None)
            desire.append(None)
            action.append(None)
    else:
        # Append None to all lists as a placeholder for failed extraction
        attention.append(None)
        interest.append(None)
        desire.append(None)
        action.append(None)



Error: Regex search failed for text: Attention: 85%
Interest: 85%
Desire: 80%
Action: 85%
Error: Regex search failed for text: * Attention: 8/10
Error: Regex search failed for text: --- legs

This product description effectively captures the attention of a customer as it highlights the key features of the boots, such as the wide waistband and flared calf, which are likely to appeal to the customer. Additionally, the use of fashionable language and references to elongating the legs and adding sophistication to the look may pique the customer's interest.

However, it is not clear how well this description is able to cultivate desire in the potential customer, as it does not mention any specific benefits or reasons why the customer should want these boots. Additionally, the inclusion of additional products as "similar to this product" may not be effective in prompting the customer to take action, as it could be distracting or overwhelming.

Overall, this product description has a mixed le

In [255]:
data = {
    'ItemSku': unique_values,
    'content': content,
    'cosine_similarity': cosine_similarity,
    'flesch_kincaid_grade_level': flesch_kincaid_grade_level,
    'gunning_fog_index': gunning_fog_index,
    'attention': attention,
    'interest': interest,
    'desire': desire,
    'action': action,
    'confidence_scores': confidence_scores
}

# Create DataFrame
df = pd.DataFrame(data)

In [250]:

# Define function to determine pass/fail
def pass_fail(row):
    if (row['cosine_similarity'] > 0.4 and
        row['flesch_kincaid_grade_level'] < 15 and
        row['gunning_fog_index'] < 20 and
        row['attention'] > 5 and
        row['interest'] > 5 and
        row['desire'] > 5 and
        row['action'] > 5):
        return 'Pass'
    else:
        return 'Fail'

# Apply the function to create 'Pass/Fail' column
df['Pass/Fail'] = df.apply(pass_fail, axis=1)


In [252]:
updated_df = pd.merge(sku_df, df[['ItemSku', 'Pass/Fail']], on='ItemSku', how='left')

# Fill missing values in 'Pass/Fail' column with 'Fail'
updated_df['Pass/Fail'].fillna('Fail', inplace=True)